## Парсинг табличек по поиску - явно указан ФУ:

### Название|Ссылка на статью|Авторы|Цитирования(количество)

In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# ДАННЫЕ ДЛЯ ВХОДА 
USERNAME = ""
PASSWORD = ""

# Настройки Selenium
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

# Установка и запуск Chrome WebDriver
service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service, options=chrome_options)

# Вход в eLibrary (доступ к поиску есть только у зарегистрированных пользователей)
#url = "https://www.elibrary.ru/defaultx.asp"
#browser.get(url)
#time.sleep(5)
#
## Заполняем логин и пароль
#browser.find_element(By.ID, "login").send_keys(USERNAME)
#browser.find_element(By.ID, "password").send_keys(PASSWORD)
#browser.find_element(By.ID, "password").send_keys(Keys.RETURN)
#time.sleep(10)  # Ожидание загрузки страницы

# URL страницу расширенного поиска
base_url = "https://elibrary.ru/org_items.asp?orgsid=1074"
browser.get(base_url)
# за это время в ручную выбираются настройки поиска
time.sleep(300)

# Цикл по страницам (всего 52 страницы)
for page_num in range(1, 47):
    print(f"Обработка страницы {page_num}...")
    
    # Получаем HTML-код текущей страницы
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Парсим данные
    publications = []
    
    for row in soup.find_all("tr", id=lambda x: x and x.startswith("arw")):
        link_tag = row.find("a", href=True)
        title = link_tag.text.strip() if link_tag else ""
        article_link = "https://elibrary.ru" + link_tag["href"] if link_tag else ""
        
        authors_tag = row.find("i")
        authors = authors_tag.text.strip() if authors_tag else ""
        
        # Парсим количество цитирований
        citations_tag = row.find("td", {"align": "center", "valign": "middle", "class": "select-tr-right"})
        citations = citations_tag.text.strip() if citations_tag else "0"
        
        publications.append({
            "Название": title,
            "Ссылка на статью": article_link,
            "Авторы": authors,
            "Цитирования": citations
        })
    
    # Создаем и сохраняем датафрейм для текущей страницы
    df = pd.DataFrame(publications)
    df.to_csv(f"publications_page_{page_num}.csv", index=False, encoding="utf-8")
    print(f"Страница {page_num} сохранена в publications_page_{page_num}.csv")
    
    # Если это не последняя страница, переходим на следующую
    if page_num < 52:
        try:
            # Находим кнопку перехода на следующую страницу
            next_page_button = browser.find_element(By.XPATH, f'//a[@title="Следующая страница" and contains(@href, "goto_page({page_num + 1})")]')
            next_page_button.click()
            time.sleep(30)  # Ожидание загрузки новой страницы
        except Exception as e:
            print(f"Не удалось перейти на страницу {page_num + 1}: {e}")
            break

# Закрываем браузер
browser.quit()
print("Сбор данных завершен.")

## Парсинг остальной информации по прямым ссылкам

In [ ]:
import os
import re
import time
import glob
import random
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from parser.parse_en_block import extract_article_data
from parser.parse_main_block import extract_journal_info, extract_article_title, extract_keywords, extract_abstract, extract_authors, extract_article_info, extract_rubric

# ДАННЫЕ ДЛЯ ВХОДА 
USERNAME = "rozhkrozhk"
PASSWORD = "rozhkrozhk1"

# Список User-Agent
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.198 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:117.0) Gecko/20100101 Firefox/117.0",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 16_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (iPad; CPU OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 13; Pixel 6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.60 Mobile Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/117.0",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.140 Safari/537.36"
]

# Настройки Selenium
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_argument(f"user-agent={user_agents[0]}")  # начальный UA

# Установка и запуск Chrome WebDriver
service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service, options=chrome_options)

# Скрываем webdriver в JS
browser.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": "Object.defineProperty(navigator, 'webdriver', { get: () => undefined })"
})

# Вход в eLibrary
browser.get("https://www.elibrary.ru/")
time.sleep(5)
browser.find_element(By.ID, "login").send_keys(USERNAME)
browser.find_element(By.ID, "password").send_keys(PASSWORD)
browser.find_element(By.ID, "password").send_keys(Keys.RETURN)
time.sleep(10)

# Список CSV-файлов
START_N = 20
END_N = 47
csv_files = [f"data/base/publications_page_{i}.csv" for i in range(START_N, END_N)]

for i, csv_file in enumerate(csv_files):
    i += START_N
    df = pd.read_csv(csv_file)
    print(f"Обработка файла {i}/{len(csv_files)}: {csv_file}")
    articles = []
    journals = []

    for j, link in enumerate(tqdm(df['Ссылка на статью'])):
        try:
            # Меняем user-agent каждые 10 запросов через CDP
            if j % 10 == 0:
                ua_index = (j // 10) % len(user_agents)
                new_user_agent = user_agents[ua_index]
                browser.execute_cdp_cmd("Network.setUserAgentOverride", {
                    "userAgent": new_user_agent
                })
                print(f"[INFO] Сменён User-Agent на: {new_user_agent}")

            browser.get(link)
            time.sleep(random.randint(1, 10))

            if "captcha" in browser.page_source.lower():
                input("Капча! Реши и нажми Enter...")

            soup = BeautifulSoup(browser.page_source, "html.parser")

            # elibrary_id и DOI
            elibrary_id = None
            doi = None
            elibrary_id_link = soup.find('a', href=re.compile(r'item\.asp\?id=\d+'))
            doi_link = soup.find('a', href=re.compile(r'https://doi\.org/.*'))

            if elibrary_id_link:
                elibrary_id = elibrary_id_link.string.strip()
            if doi_link:
                doi = doi_link.string.strip()

            journal_name, publisher, issn, eissn = extract_journal_info(soup)
            article_type, language, year = extract_article_info(soup)
            rubric_oecd = extract_rubric(soup)

            title_en = key_words_en = abstract_en = authors_metadata_en = None
            title_ru = key_words_ru = abstract_ru = authors_metadata_ru = None

            if language == 'английский' or language == 'не определен':
                title_en = extract_article_title(soup)
                key_words_en = extract_keywords(soup)
                abstract_en = extract_abstract(soup)
                authors_metadata_en = extract_authors(soup)
            else:
                title_ru = extract_article_title(soup)
                key_words_ru = extract_keywords(soup)
                abstract_ru = extract_abstract(soup)
                authors_metadata_ru = extract_authors(soup)

                eng_data = extract_article_data(soup)
                if eng_data is None:
                    title_en, key_words_en, abstract_en, authors_metadata_en = "", [], "", []
                else:
                    title_en, key_words_en, abstract_en, authors_metadata_en = eng_data

            journal_metadata = {
                "Ссылка на статью": link,
                "DOI": doi,
                "Название": journal_name,
                "Издательство": publisher,
                "ISSN": issn,
                "eISSN": eissn,
            }

            article_metadata = {
                "Ссылка на статью": link,
                "DOI": doi,
                "eLIBRARY ID": elibrary_id,
                "article_type": article_type,
                "year": year,
                "language": language,
                "title_en": title_en,
                "key_words_en": key_words_en,
                "abstract_en": abstract_en,
                "authors_metadata_en": authors_metadata_en,
                "title_ru": title_ru,
                "key_words_ru": key_words_ru,
                "abstract_ru": abstract_ru,
                "authors_metadata_ru": authors_metadata_ru,
                "OECD": rubric_oecd
            }

            articles.append(article_metadata)
            journals.append(journal_metadata)

        except Exception as e:
            print(f"Ошибка при обработке ссылки {link}: {e}")
            continue

    articles_df = pd.DataFrame(articles)
    journals_df = pd.DataFrame(journals)
    articles_df.to_csv(f"data/articles/articles_page_{i}.csv", index=False, encoding="utf-8")
    journals_df.to_csv(f"data/journals/journals_page_{i}.csv", index=False, encoding="utf-8")
    print(f"Файл articles_page_{i}.csv и journal_page_{i}.csv сохранены.")

# Закрываем браузер
browser.quit()


In [4]:
articles_df = pd.DataFrame(articles)
journals_df = pd.DataFrame(journals)
articles_df.to_csv(f"data/articles/articles_page_{i}_1.csv", index=False, encoding="utf-8")
journals_df.to_csv(f"data/journals/journals_page_{i}_1.csv", index=False, encoding="utf-8")
print(f"Файл articles_page_{i}_1.csv и journal_page_{i}_1.csv сохранены.")

Файл articles_page_17_1.csv и journal_page_17_1.csv сохранены.
